In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/01/30 15:50:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
green_folderpath = 'data/pq/green/*/*'
yellow_folderpath = 'data/pq/yellow/*/*'

df_green = spark.read.parquet(green_folderpath)
df_yellow = spark.read.parquet(yellow_folderpath)

In [4]:
df_green.createOrReplaceTempView('green')
df_yellow.createOrReplaceTempView('yellow')

In [5]:
df_green_revenue = spark.sql(
    """
    SELECT
        DATE_TRUNC('HOUR', lpep_pickup_datetime) AS hour,
        PULocationID AS zone,
        SUM(total_amount) AS amount,
        COUNT(1) AS records_cnt
    FROM
        green
    GROUP BY
        1, 2
    """
)

df_yellow_revenue = spark.sql(
    """
    SELECT
        DATE_TRUNC('HOUR', tpep_pickup_datetime) AS hour,
        PULocationID AS zone,
        SUM(total_amount) AS amount,
        COUNT(1) AS records_cnt
    FROM
        yellow
    GROUP BY
        1, 2
    """
)

In [7]:
green_revenue_folderpath = 'data/report/revenue/green'
yellow_revenue_folderpath = 'data/report/revenue/yellow'

df_green_revenue \
    .repartition(20) \
    .write.parquet(green_revenue_folderpath, mode='overwrite')

df_yellow_revenue \
    .repartition(20) \
    .write.parquet(yellow_revenue_folderpath, mode='overwrite')